## Enregistre des images durant 5 secondes avec laps

In [1]:
import os
import numpy as np
import cv2
import datetime
import time
import glob

class CFEVideoConf(object):
    # Standard Video Dimensions Sizes
    STD_DIMENSIONS =  {
        "360p": (480, 360),
        "480p": (640, 480),
        "720p": (1280, 720),
        "1080p": (1920, 1080),
        "4k": (3840, 2160),
    }
    # Video Encoding, might require additional installs
    # Types of Codes: http://www.fourcc.org/codecs.php
    VIDEO_TYPE = {
        'avi': cv2.VideoWriter_fourcc(*'XVID'),
        #'mp4': cv2.VideoWriter_fourcc(*'H264'),
        'mp4': cv2.VideoWriter_fourcc(*'XVID'),
    }

    width           = 640
    height          = 480
    dims            = (640, 480)
    capture         = None
    video_type      = None
    def __init__(self, capture, filepath, res="480p", *args, **kwargs):
        self.capture = capture
        self.filepath = filepath
        self.width, self.height = self.get_dims(res=res)
        self.video_type = self.get_video_type()

    # Set resolution for the video capture
    # Function adapted from https://kirr.co/0l6qmh
    def change_res(self, width, height):
        self.capture.set(3, width)
        self.capture.set(4, height)

    def get_dims(self, res='480p'):
        width, height = self.STD_DIMENSIONS['480p']
        if res in self.STD_DIMENSIONS:
            width, height = self.STD_DIMENSIONS[res]
        self.change_res(width, height)
        self.dims = (width, height)
        return width, height

    def get_video_type(self):
        filename, ext = os.path.splitext(self.filepath)
        if ext in self.VIDEO_TYPE:
          return  self.VIDEO_TYPE[ext]
        return self.VIDEO_TYPE['avi']

In [2]:
timelapse_img_dir = "images/"
if not os.path.exists(timelapse_img_dir):
    os.mkdir(timelapse_img_dir)

seconds_duration = 25
laps = .25
now = datetime.datetime.now()
finish_time = now + datetime.timedelta(seconds=seconds_duration)

cap = cv2.VideoCapture(0)
save_path = "dossier/timelaps.mp4"
frames_per_seconds = 24
# config = CFEVideoConf(save_path, config.video_type, frames_per_seconds, config.dims)
config = CFEVideoConf(cap, filepath=save_path, res='720p')
out=cv2.VideoWriter(save_path, config.video_type,frames_per_seconds, config.dims )

i=0
while datetime.datetime.now() < finish_time:
    ret, frame = cap.read()
    filename = f"{timelapse_img_dir}/{i}.jpg"
    i += 1
    cv2.imwrite(filename, frame)
#     out.write(frame)
    cv2.imshow('frame', frame)
    time.sleep(laps)
    if cv2.waitKey(20) & 0xff == ord('q'):
        break

Clear_img = True        

def images_to_video(out, img_dir, clear_img=True):
    image_list = glob.glob(f"{img_dir}/*.jpg")
    sorted_img = sorted(image_list, key = os.path.getmtime)
    for file in sorted_img:
        image_frame = cv2.imread(file)
        out.write(image_frame)


    if Clear_img:
        for file in image_list:
           os.remove(file) 

images_to_video(out, timelapse_img_dir, clear_img=True)
cap.release()
out.release()
cv2.destroyAllWindows()